# Connect to server and performe command sending

## Import the strategy

In [1]:
from strategy_training import *

## Server tools

In [2]:
class Tools:
    @staticmethod
    def update_board_matrix(board_matrix, agent_state):
        board_info = list(agent_state['map_commands_raw'])
        for i in range(len(board_info)):
            if i % 5 == 0:
                temp_cell = [board_info[i + 2], board_info[i + 3], board_info[i + 4]]
                #             humans             vampires            werewolves
                board_matrix[board_info[i + 1]][board_info[i]] = temp_cell
        return board_matrix
    
    @staticmethod
    def board2points(board_matrix):
        """
        Convert board_matrix to 3 lists of Point object: Vampires, Werewolves, Humans.
        :param board_matrix:
        
        :return list_creatures: list of Point
            list_vampires
            list_werewolves
            list_humans
        """
        human_groups_positon = np.argwhere(board_matrix[:,:, 0] > 0)
        vampire_groups_positon = np.argwhere(board_matrix[:,:, 1] > 0)
        werewolf_groups_positon = np.argwhere(board_matrix[:,:, 2] > 0)
        
        list_humans = []
        list_vampires = []
        list_werewolves = []
        
        for humans_position in human_groups_positon:
            x_position = humans_position[0]
            y_position = humans_position[1]
            population = board_matrix[x_position, y_position, 0]
            list_humans.append(Point(x_position, y_position, population))
        
        for vampires_position in vampire_groups_positon:
            x_position = vampires_position[0]
            y_position = vampires_position[1]
            population = board_matrix[x_position, y_position, 1]
            list_vampires.append(Point(x_position, y_position, population))
            
        for werewolves_position in werewolf_groups_positon:
            x_position = werewolves_position[0]
            y_position = werewolves_position[1]
            population = board_matrix[x_position, y_position, 2]
            list_werewolves.append(Point(x_position, y_position, population))
        
        return list_vampires, list_werewolves, list_humans
    
    @staticmethod
    def identify_species(board_matrix, agent_state):
        x_initial = agent_state['start_position'][0]
        y_initial = agent_state['start_position'][1]
        temp_cell = board_matrix[y_initial][x_initial]

        if temp_cell[1] > 0:
            species_dict = {'our_species': 'vampires', 'opponent_species': 'werewolves'}
        elif temp_cell[2] > 0:
            species_dict = {'our_species': 'werewolves', 'opponent_species': 'vampires'}
        else:
            raise NameError("Can't guess our species")

        return species_dict

## Server communication

### Dependencies

In [3]:
import socket
import sys
import struct
import os
import numpy as np
import tools
import time

In [4]:
class Server:
    def __init__(self, host, port):
        print("Creating connection to server with socket")
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.sock.connect((host, port))
        print("Creating agent_state to get playground")
        self.agent_state = {'height': None,
                            'width': None,
                            'number_of_homes': None,
                            'homes_raw': None,
                            'start_position': None,
                            'number_map_commands': None,
                            'map_commands_raw': None}

        # First Send the NME Message to Server
        print('Starting the Game by sending NME and team name')
        self.send_nme('NME', 'Team6')
        # Receiving SET, HUM, HME, MAP, UPD, END, BYE
        self.get_set()
        self.get_hum()
        self.get_hme()
        self.get_map()
        # Creating a map array board matrix
        self.set_board_matrix()
        # Guess if we are Vampire or Werewolves
        self.species = self.perform_species_guess()

    def perform_species_guess(self):
        """
        Display if we are vampires ou werewolves using tools
        :return:
        """
        species_dict = Tools.identify_species(self.board_matrix, self.agent_state)
        if species_dict['our_species'] == 'vampires':
            print('We are vampires')
            return "V"
        elif species_dict['our_species'] == 'werewolves':
            print('We are werewolves')
            return "W"
        else:
            raise NameError("I don't recognize this species" + str(species_dict['our_species']))

    @staticmethod
    def start_server_ex(server_path):
        """
        Try to run executable server program
        :param server_path:
        :return:
        """
        print("Starting server from " + server_path)
        os.system('start ' + server_path)

    def send_nme(self, nme_command, msg):
        """
        Send NME command to server with team name
        :param nme_command:
        :param msg:
        :return:
        """
        print('Sending the NME Message to the Server')
        self.sock.send(nme_command.encode("ascii"))
        self.sock.send(struct.pack("1B", len(msg)))
        self.sock.send(msg.encode("ascii"))

    def get_set(self):
        """
        Getting playground size with "SET" command (setting the map)
        :return:
        """
        print("SET: Getting playground size")
        header = self.sock.recv(3).decode("ascii")
        if header != "SET":
            print("Protocol Error at SET")
        else:
            (self.agent_state['height'], self.agent_state['width']) = self.receive_data(2, "2B")

    def get_hum(self):
        """
        Getting human houses positions
        :return:
        """
        print("HUM: Getting info on human an houses")
        header = self.sock.recv(3).decode("ascii")
        if header != "HUM":
            print("Protocol Error at HUM")
        else:
            self.agent_state['number_of_homes'] = self.receive_data(1, "1B")[0]
            self.agent_state['homes_raw'] = \
                self.receive_data(
                    self.agent_state['number_of_homes'] * 2,
                    "{}B".format(self.agent_state['number_of_homes'] * 2)
                )

    def get_hme(self):
        """
        Getting start position
        :return:
        """
        print("HME: Getting our start position")
        header = self.sock.recv(3).decode("ascii")
        if header != "HME":
            print("Protocol Error at HME")
        else:
            self.agent_state['start_position'] = tuple(self.receive_data(2, "2B"))

    def get_map(self):
        """
        Getting map
        :return:
        """
        print("MAP: Getting map first state")
        header = self.sock.recv(3).decode("ascii")
        if header != "MAP":
            print("Protocol Error at MAP")
        else:
            self.update_agent_state()

    def set_board_matrix(self):
        print("Creating a Numpy Array for Board.")
        #self.board_cell_dict = {'cell_human_count': 0, 'cell_vampire_count': 0, 'cell_werewolves_count': 0}
        # self.board_cell_dict = [0, 0, 0]
        #                       H  V  W
        self.board_matrix = np.zeros(shape=(self.agent_state['height'], self.agent_state['width'], 3))
        # print("Board matrix is at first:" + str(self.board_matrix))

        print("Try board matrix update")
        self.board_matrix = Tools.update_board_matrix(self.board_matrix, self.agent_state)

    def receive_data(self, size, fmt):
        data = bytes()
        while len(data) < size:
            data += self.sock.recv(size - len(data))
        return struct.unpack(fmt, data)

    def send_move(self, migration):
        ## Testing the Move command. Right now just making few random moves.
        self.sock.send("MOV".encode("ascii"))
        
        origin_position = migration.origin_position
        population = migration.population
        target_position = migration.target_position
        print("Performing move", migration)
        # Choosing to perform 1 command
        self.sock.send(struct.pack("1B", 1))
        # Incoming cell
        print("In server referential, making the move : X=", origin_position.y, "Y=", origin_position.x)
        print("To : X=", target_position.y, "Y=", target_position.x)
        print("With N=", population)
        self.sock.send(struct.pack("2B", origin_position.y, origin_position.x))
        # 2 vampires
        self.sock.send(struct.pack("1B", population))
        # Destination cell
        self.sock.send(struct.pack("2B", target_position.y, target_position.x))

    def update_agent_state(self):
        self.agent_state['number_map_commands'] = self.receive_data(1, "1B")[0]
        self.agent_state['map_commands_raw'] = \
            self.receive_data(
                self.agent_state['number_map_commands'] * 5,
                "{}B".format(self.agent_state['number_map_commands'] * 5)
            )

    def update_board_matrix(self):
        self.board_matrix = Tools.update_board_matrix(self.board_matrix, self.agent_state)

## Init connection

In [5]:
def playing_game(srv):
    # send_move(sock,'MOV')
    while True:
        # UPD---------------------
        header = srv.sock.recv(3).decode("ascii")
        if header == "UPD":
            print('A UPD Packet has been received')
            srv.update_agent_state()
            # print(srv.agent_state)
            print('------------------------------------------------------------------------------------------')
            
            # Update the matrix
            srv.update_board_matrix()
            # Get it
            current_board_matrix = srv.board_matrix
            
            # Get the creatures Point lists
            list_vampires, list_werewolves, list_humans = Tools.board2points(board_matrix=current_board_matrix)
            
            print("vampires", list_vampires)
            # Get the best move to perform
            best_move_migration = interface_strategy(
                width=srv.agent_state['width'], 
                height=srv.agent_state['height'], 
                list_vampires=list_vampires, 
                list_werewolves=list_werewolves,
                list_humans=list_humans, 
                our_species=srv.species, 
                max_depth=6, 
                our_name="Dracula", 
                enemy_name="Garou", 
                verbose=0)
            print("We should do", best_move_migration)
            
            srv.send_move(best_move_migration)
            time.sleep(5)
            
            ### PLAY !!!!!!

        # END--------------------------------------------------------------------------------------------------------------
        elif header == "END":
            print('The END Packet has been received')
            print("The Game has ended")
            print("Closing the connection now")
            srv.sock.close()
            sys.exit()

        # BYE------------------------------------------------------------------------------------------------------------
        elif header == "BYE":
            print('BYE Packet has been received')
            print("Closing the connection now")
            srv.sock.close()

In [6]:
SERVER_PROGRAM_PATH = 'game-intelligence\\Resources\\VampiresVSWerewolvesGameServer'


def main():
    # server.start_server(SERVER_PROGRAM_PATH)
    host = 'localhost'
    port = 5555
    srv = Server(host, port)
    print("Width:", srv.agent_state['width'])
    print("Height:", srv.agent_state['height'])
    playing_game(srv)


## Unleashing hell

In [7]:
main()

Creating connection to server with socket
Creating agent_state to get playground
Starting the Game by sending NME and team name
Sending the NME Message to the Server
SET: Getting playground size
HUM: Getting info on human an houses
HME: Getting our start position
MAP: Getting map first state
Creating a Numpy Array for Board.
Try board matrix update
We are vampires
Width: 10
Height: 5
A UPD Packet has been received
------------------------------------------------------------------------------------------
vampires [3_4n4.0]
Dracula is playing vampires
Garou is playing werewolves
We should do Moving 4 from 3_4 to 2_3
Performing move Moving 4 from 3_4 to 2_3
In server referential, making the move : X= 4 Y= 3
To : X= 3 Y= 2
With N= 4
A UPD Packet has been received
------------------------------------------------------------------------------------------
vampires [2_3n4.0]
Dracula is playing vampires
Garou is playing werewolves
We should do Moving 4 from 2_3 to 1_2
Performing move Moving 4 f

KeyboardInterrupt: 